# Lab 4, Group 21

### Question 1. a

In [11]:
from nltk.corpus.reader import ConllCorpusReader
train = ConllCorpusReader('./CONLL2003', 'train.txt', ['words', 'pos', 'ignore', 'chunk'])
train_file = '/CONLL2003/train.txt'

training_features = []
training_gold_labels = []
for token, pos, ne_label in train.iob_words():
    a_dict = {
        'word': token,
        'pos': pos
    }
        # Append the features dictionary to the training_features list
    training_features.append(a_dict)
    
    # Append the NE label to the training_gold_labels list
    training_gold_labels.append(ne_label)
    

In [12]:
training_features, training_gold_labels

([{'word': 'EU', 'pos': 'NNP'},
  {'word': 'rejects', 'pos': 'VBZ'},
  {'word': 'German', 'pos': 'JJ'},
  {'word': 'call', 'pos': 'NN'},
  {'word': 'to', 'pos': 'TO'},
  {'word': 'boycott', 'pos': 'VB'},
  {'word': 'British', 'pos': 'JJ'},
  {'word': 'lamb', 'pos': 'NN'},
  {'word': '.', 'pos': '.'},
  {'word': 'Peter', 'pos': 'NNP'},
  {'word': 'Blackburn', 'pos': 'NNP'},
  {'word': 'BRUSSELS', 'pos': 'NNP'},
  {'word': '1996-08-22', 'pos': 'CD'},
  {'word': 'The', 'pos': 'DT'},
  {'word': 'European', 'pos': 'NNP'},
  {'word': 'Commission', 'pos': 'NNP'},
  {'word': 'said', 'pos': 'VBD'},
  {'word': 'on', 'pos': 'IN'},
  {'word': 'Thursday', 'pos': 'NNP'},
  {'word': 'it', 'pos': 'PRP'},
  {'word': 'disagreed', 'pos': 'VBD'},
  {'word': 'with', 'pos': 'IN'},
  {'word': 'German', 'pos': 'JJ'},
  {'word': 'advice', 'pos': 'NN'},
  {'word': 'to', 'pos': 'TO'},
  {'word': 'consumers', 'pos': 'NNS'},
  {'word': 'to', 'pos': 'TO'},
  {'word': 'shun', 'pos': 'VB'},
  {'word': 'British', 'pos

#### 1. b

In [20]:
from collections import Counter 

test = ConllCorpusReader('./CONLL2003', 'test.txt', ['words', 'pos', 'ignore', 'chunk'])
test_features = []
test_gold_labels = []
for token, pos, ne_label in test.iob_words():
    a_dict = {
        'word': token,
        'pos': pos
    }
    test_features.append(a_dict)
    test_gold_labels.append(ne_label)

num_train_instances = len(training_features)
num_test_instances = len(test_features)

train_labels_freq = Counter(training_gold_labels)
test_labels_freq = Counter(test_gold_labels)

#### b) i) and ii)

In [21]:
num_train_instances, num_test_instances, train_labels_freq, test_labels_freq

(203621,
 46435,
 Counter({'O': 169578,
          'B-LOC': 7140,
          'B-PER': 6600,
          'B-ORG': 6321,
          'I-PER': 4528,
          'I-ORG': 3704,
          'B-MISC': 3438,
          'I-LOC': 1157,
          'I-MISC': 1155}),
 Counter({'O': 38323,
          'B-LOC': 1668,
          'B-ORG': 1661,
          'B-PER': 1617,
          'I-PER': 1156,
          'I-ORG': 835,
          'B-MISC': 702,
          'I-LOC': 257,
          'I-MISC': 216}))

#### b) iii)
The datasets are not balanced with respect to the distribution of NERC labels. The majority of instances are tagged as 'O', indicating they are not part of a named entity. 

The training and test data are consistent in terms of the relative frequencies of different labels However, the exact counts of each label differ due to the different sizes of the datasets.

#### c) 

In [23]:
from sklearn.feature_extraction import DictVectorizer

combined_features = training_features + test_features
vectorizer = DictVectorizer(sparse=True)
vectorized_features = vectorizer.fit_transform(combined_features)

train_vectorized = vectorized_features[:num_train_instances]
test_vectorized = vectorized_features[num_train_instances:]

train_vectorized.shape, test_vectorized.shape


((203621, 27361), (46435, 27361))

#### d) 

- The SVM classifier performs exceptionally well on non-entity ('O') labels due to their predominance in the dataset, showing high precision and recall. 

- It struggles with 'I-PER' labels, likely due to over-predicting this category, and shows lower recall for 'B-PER' and 'B-ORG', indicating missed instances of these entities. These variations suggest the model's performance is influenced by the imbalance in label distribution and the distinctiveness of entity patterns in the training data.

In [24]:
from sklearn import svm
from sklearn.metrics import classification_report

lin_clf = svm.LinearSVC()
lin_clf.fit(train_vectorized, training_gold_labels)
test_predictions = lin_clf.predict(test_vectorized)

classification_report_test = classification_report(test_gold_labels, test_predictions)


/home/temp/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


'              precision    recall  f1-score   support\n\n       B-LOC       0.81      0.78      0.79      1668\n      B-MISC       0.78      0.66      0.72       702\n       B-ORG       0.79      0.52      0.63      1661\n       B-PER       0.86      0.44      0.58      1617\n       I-LOC       0.62      0.53      0.57       257\n      I-MISC       0.57      0.59      0.58       216\n       I-ORG       0.70      0.47      0.56       835\n       I-PER       0.33      0.87      0.48      1156\n           O       0.98      0.98      0.98     38323\n\n    accuracy                           0.92     46435\n   macro avg       0.72      0.65      0.65     46435\nweighted avg       0.94      0.92      0.92     46435\n'

In [26]:
print(classification_report_test)


              precision    recall  f1-score   support

       B-LOC       0.81      0.78      0.79      1668
      B-MISC       0.78      0.66      0.72       702
       B-ORG       0.79      0.52      0.63      1661
       B-PER       0.86      0.44      0.58      1617
       I-LOC       0.62      0.53      0.57       257
      I-MISC       0.57      0.59      0.58       216
       I-ORG       0.70      0.47      0.56       835
       I-PER       0.33      0.87      0.48      1156
           O       0.98      0.98      0.98     38323

    accuracy                           0.92     46435
   macro avg       0.72      0.65      0.65     46435
weighted avg       0.94      0.92      0.92     46435



#### e) 